## Preprocess data to get training and testing data

I,II,III,aVL,aVR,aVF,V1–V6

In [1]:
# Import package
import time
import numpy as np
import wfdb
import ast
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import mpl
from scipy.fftpack import fft, ifft 
from scipy import signal
# from biosppy.signals import ecg
import neurokit2 as nk
from sklearn import *
from collections import OrderedDict

In [2]:
#Set the read file path
path = '/global/D1/homes/jayao/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.2/ptbxl/'

X = np.load(path + 'raw100_sex1.npy', allow_pickle=True)
sampling_rate = 100

In [3]:
df = pd.read_csv(path+'ptbxl_database.csv')

In [4]:
# Read the file and convert tags
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [5]:
X.shape

(10445, 1000, 12)

In [6]:
# Get diagnostic information in scp_statements.csv
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)

In [7]:
agg_df = agg_df[agg_df.diagnostic == 1]

In [8]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [9]:
#Add diagnostic information
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [14]:
# Filter dataframe Y to include only rows where sex=1
Y_sex1 = Y[Y['sex'] == 1]

In [15]:
Y_sex1.shape

(10445, 44)

In [16]:
# Split data into train and test
test_fold = 10
# # Train
X_train = X[(Y_sex1.strat_fold <= 8)]
y_train = Y_sex1[(Y_sex1.strat_fold <= 8)].diagnostic_superclass
# # Test
X_test = X[(Y_sex1.strat_fold >8)]
y_test = Y_sex1[(Y_sex1.strat_fold > 8)].diagnostic_superclass


print(X_train.shape, y_train.shape)
print(X_test.shape,  y_test.shape)

(8329, 1000, 12) (8329,)
(2116, 1000, 12) (2116,)


In [17]:
save_path = '/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/data/sex1/'

np.save(save_path+'x_train.npy', X_train)
np.save(save_path+'y_train.npy', np.array(y_train))
np.save(save_path+'x_test.npy', X_test)
np.save(save_path+'y_test.npy', np.array(y_test))

## ECG SHaP starts

In [1]:
from tensorflow.keras import layers, optimizers, losses, metrics, activations, regularizers, callbacks
from keras.models import Model
import numpy as np
import pandas as pd
# from tensorflow.keras.layers import LSTM

In [2]:
# path = "/global/D1/homes/jayao/XAI-Based-ECG-Diagnostics-main/data/sex1/"
# path = "/D:/Test Jupyter/ECG-Classfier-main/code/Test ECG/sex1/"
# Define the path to the directory
path = 'D:\\Test Jupyter\\ECG-Classfier-main\\code\\Test ECG\\sex1\\'


x_train = np.load(path + 'x_train.npy')
y_train = np.load(path + 'y_train.npy', allow_pickle=True)
x_test  = np.load(path + 'x_test.npy')
y_test  = np.load(path + 'y_test.npy', allow_pickle=True)
print(x_train.shape)

(8329, 1000, 12)


In [3]:
x_train = x_train.transpose(0, 2, 1)            # transpose working correctly
x_test  = x_test.transpose(0, 2, 1)
print(x_train.shape)
print(x_test.shape)

(8329, 12, 1000)
(2116, 12, 1000)


In [4]:
x_train = x_train.reshape(8329, 12, 1000, 1)   # Add another channel
x_test  = x_test.reshape(2116, 12, 1000, 1)

In [5]:
print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test  :", x_test.shape)
print("y_test  :", y_test.shape)
print('Data loaded')

# Old OUTPUTS:
# (19601, 1000, 12)
# (19601, 12, 1000)
# x_train : (19601, 12, 1000, 1)
# y_train : (19601,)
# x_test  : (2198, 12, 1000, 1)
# y_test  : (2198,)
# Data loaded

x_train : (8329, 12, 1000, 1)
y_train : (8329,)
x_test  : (2116, 12, 1000, 1)
y_test  : (2116,)
Data loaded


In [6]:
x_test.shape

(2116, 12, 1000, 1)

In [7]:

from sklearn.preprocessing import MultiLabelBinarizer
# Convert multi-label target labels to one-hot encoded matrix
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)
print("Classes:", mlb.classes_)

Classes: ['CD' 'HYP' 'MI' 'NORM' 'STTC']


In [8]:
value_at_index = y_train[6666]
print(value_at_index)

[0 0 1 0 0]


In [9]:
y_train.shape

(8329, 5)

In [10]:
y_test.shape

(2116, 5)

In [18]:
# # ST-CNN
# # Main Version
# input = layers.Input(shape=(12, 1000, 1))

# X = layers.Conv2D(filters=32, kernel_size=(1, 5))(input)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

# convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7))(X)

# X = layers.Conv2D(filters=32, kernel_size=(1, 5))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

# convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6))(convC1)

# X = layers.Conv2D(filters=64, kernel_size=(1, 5))(X)
# X = layers.BatchNormalization()(X)
# X = layers.Add()([convC2, X])           # skip Connection
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

# convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4))(X)

# X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

# convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5))(convE1)

# X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
# X = layers.BatchNormalization()(X)
# X = layers.Add()([convE2, X])         # skip Connection
# X = layers.ReLU()(X)
# X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)
# print('Added 5 layers for temporal analysis')

# X = layers.Conv2D(filters=64, kernel_size=(12, 1))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.GlobalAveragePooling2D()(X)
# print('Added 1 layer for spatial Analysis')

# X = layers.Flatten()(X)
# print(X.shape)

# X = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.Dropout(rate=0.1)(X)

# X = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(X)
# X = layers.BatchNormalization()(X)
# X = layers.ReLU()(X)
# X = layers.Dropout(rate=0.15)(X)
# print('Added 2 fully connected layers')

# output = layers.Dense(5, activation='sigmoid')(X)
# model = Model(inputs=input, outputs=output)
# print(model.summary())

Added 5 layers for temporal analysis
Added 1 layer for spatial Analysis
(None, 64)
Added 2 fully connected layers


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12, 1000,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 12, 996,   │        192 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 12, 996,   │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 12, 996,   │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 12, 995,   │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 991,   │      5,152 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 991,   │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 12, 991,   │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 988,   │          0 │ re_lu_1[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 12, 989,   │     14,400 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 12, 984,   │     10,304 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 12, 984,   │     24,640 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 984,   │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 12, 984,   │          0 │ conv2d_3[0][0],   │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 12, 984,   │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 12, 983,   │          0 │ re_lu_2[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 12, 981,   │     12,352 │ max_pooling2d_2[

 Total params: 166,085 (648.77 KB)

 Trainable params: 165,061 (644.77 KB)

 Non-trainable params: 1,024 (4.00 KB)

None


Hyper parameter tuning:
EarlyStopping monitors a specified metric, here: "val_loss"
If the val_loss does not improve for a certain number of epochs defined by patience (in this case, 6 epochs), training is stopped early.
The restore_best_weights=True argument ensures that the weights of the model are restored to the best weights when training stopped.

Learning Rate Reduction:
ReduceLROnPlateau monitors the validation loss ("val_loss").
If the validation loss does not improve for a certain number of epochs defined by patience (in this case, 3 epochs), the learning rate is reduced by a factor defined by factor.

In [11]:
#Enhanced ST-CNN
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Model

# Main Version
input = layers.Input(shape=(12, 1000, 1))

X = layers.Conv2D(filters=32, kernel_size=(1, 5), padding='same')(input)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)

convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7), padding='same')(X)

X = layers.Conv2D(filters=32, kernel_size=(1, 5), padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1, padding='same')(X)

convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6), padding='same')(convC1)

X = layers.Conv2D(filters=64, kernel_size=(1, 5), padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
residual_1 = layers.Add()([convC2, X])           # skip Connection
X = layers.ReLU()(residual_1)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)

convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4), padding='same')(X)

X = layers.Conv2D(filters=64, kernel_size=(1, 3), padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1, padding='same')(X)

convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5), padding='same')(convE1)

X = layers.Conv2D(filters=64, kernel_size=(1, 3), padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
residual_2 = layers.Add()([convE2, X])         # skip Connection
X = layers.ReLU()(residual_2)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1, padding='same')(X)
print('Added 5 layers for temporal analysis')

# Spatial Analysis
X = layers.Conv2D(filters=64, kernel_size=(12, 1), padding='same')(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.GlobalAveragePooling2D()(X)
print('Added 1 layer for spatial Analysis')

# Fully Connected Layers
X = layers.Flatten()(X)
X = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.3)(X)

X = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.3)(X)
print('Added 2 fully connected layers')

# Output Layer
output = layers.Dense(5, activation='sigmoid')(X)

# Define the model
model = Model(inputs=input, outputs=output)
print(model.summary())


Added 5 layers for temporal analysis
Added 1 layer for spatial Analysis
Added 2 fully connected layers


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12, 1000,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 12, 1000,  │        192 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 12, 1000,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 12, 1000,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 12, 1000,  │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 1000,  │      5,152 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 1000,  │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 12, 1000,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 1000,  │          0 │ re_lu_1[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 12, 1000,  │     10,304 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 12, 1000,  │     14,400 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 1000,  │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 12, 1000,  │     24,640 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 12, 1000,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 12, 1000,  │          0 │ conv2d_3[0][0],   │
│                     │ 64)               │            │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 12, 1000,  │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 12, 1000,  │          0 │ re_lu_3[0][0]   

 Total params: 166,085 (648.77 KB)

 Trainable params: 165,061 (644.77 KB)

 Non-trainable params: 1,024 (4.00 KB)

None


In [12]:
# Source: https://keras.io/api/callbacks/
# Source: https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

early    = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reducelr = callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)
callback = [early, reducelr]

# # ST-CNN
# model.compile(optimizer = optimizers.Adam(learning_rate=0.005),
#Enhanced ST-CNN
model.compile(optimizer = optimizers.Adam(learning_rate=0.001),
              loss = losses.BinaryCrossentropy(),
              metrics = [metrics.BinaryAccuracy(), metrics.AUC(curve='ROC', multi_label=True)])

history = model.fit(x_train, y_train, validation_split=0.12, epochs=20, batch_size=64, callbacks=callback)

Epoch 1/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 1289s 11s/step - auc: 0.6734 - binary_accuracy: 0.7430 - loss: 1.4952 - val_auc: 0.5527 - val_binary_accuracy: 0.7368 - val_loss: 0.9637 - learning_rate: 0.0010
Epoch 2/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 1369s 12s/step - auc: 0.8441 - binary_accuracy: 0.8463 - loss: 0.7009 - val_auc: 0.6429 - val_binary_accuracy: 0.7378 - val_loss: 0.6927 - learning_rate: 0.0010
Epoch 3/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 1433s 12s/step - auc: 0.8779 - binary_accuracy: 0.8671 - loss: 0.4605 - val_auc: 0.8100 - val_binary_accuracy: 0.7806 - val_loss: 0.5549 - learning_rate: 0.0010
Epoch 4/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 1241s 11s/step - auc: 0.8958 - binary_accuracy: 0.8777 - loss: 0.3680 - val_auc: 0.8719 - val_binary_accuracy: 0.8140 - val_loss: 0.4607 - learning_rate: 0.0010
Epoch 5/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 1325s 12s/step - auc: 0.9025 - binary_accuracy: 0.8805 - loss: 0.3343 - val_auc: 0.8803 - val_binary_accuracy: 0.8320 - val_loss: 0.4277 - learning_rate: 0.

In [13]:

path = 'D:\\Test Jupyter\\ECG-Classfier-main\\code\\Test ECG\\model\\'
# model.save(save_path + "ST-CNN-5_final1.h5")
model.save(path + "ST-CNN-5_final1_new_sex1.h5")

In [21]:
# #ST-CNN
# y_pred_train = model.predict(x_train)
# y_pred_test  = model.predict(x_test)

# from sklearn.metrics import classification_report, precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc
# import numpy as np

# def sklearn_metrics(y_true, y_pred, mlb):
#     y_bin = np.copy(y_pred)
#     y_bin[y_bin >= 0.5] = 1
#     y_bin[y_bin < 0.5]  = 0

# #     print("y_train shape:", y_true.shape)
# # p   print("y_test shape :", y_pred.shape)


#     # Compute area under precision-Recall curve
#     auc_sum = 0
#     for i in range(y_true.shape[1]):
#         precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:, i])
#         auc_sum += auc(recall, precision)

#     print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten()) * 100))
#     print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
#     print('AUROC           : {:.2f}'.format((auc_sum / y_true.shape[1]) * 100))
#     print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))

#     # Convert binary predictions back to class labels using MultiLabelBinarizer
#     predicted_classes = mlb.inverse_transform(y_bin)

#     # Use a set to accumulate all distinct classes
#     distinct_classes = set()

#     # Iterate over predicted classes and add them to the set
#     for classes in predicted_classes:
#         distinct_classes.update(classes)

#     # Convert the set of distinct classes to a sorted list
#     class_names = sorted(list(distinct_classes))

#     # Print classification report for each class
#     print("\nClassification Report:")
#     print(classification_report(y_true, y_bin, target_names=class_names))

# # Assuming mlb is the MultiLabelBinarizer used for transforming the labels
# sklearn_metrics(y_test, y_pred_test, mlb)


261/261 ━━━━━━━━━━━━━━━━━━━━ 167s 638ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 43s 639ms/step
Accuracy        : 88.30
Macro AUC score : 91.89
AUROC           : 80.12
Micro F1 score  : 75.74

Classification Report:
              precision    recall  f1-score   support

          CD       0.76      0.69      0.73       460
         HYP       0.81      0.35      0.49       236
          MI       0.77      0.61      0.68       484
        NORM       0.82      0.90      0.86       918
        STTC       0.78      0.72      0.75       576

   micro avg       0.80      0.72      0.76      2674
   macro avg       0.79      0.65      0.70      2674
weighted avg       0.79      0.72      0.75      2674
 samples avg       0.77      0.74      0.74      2674



d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len

In [14]:
#Enhanced ST-CNN
y_pred_train = model.predict(x_train)
y_pred_test  = model.predict(x_test)

from sklearn.metrics import classification_report, precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc
import numpy as np

def sklearn_metrics(y_true, y_pred, mlb):
    y_bin = np.copy(y_pred)
    y_bin[y_bin >= 0.5] = 1
    y_bin[y_bin < 0.5]  = 0

#     print("y_train shape:", y_true.shape)
# p   print("y_test shape :", y_pred.shape)


    # Compute area under precision-Recall curve
    auc_sum = 0
    for i in range(y_true.shape[1]):
        precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:, i])
        auc_sum += auc(recall, precision)

    print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten()) * 100))
    print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
    print('AUROC           : {:.2f}'.format((auc_sum / y_true.shape[1]) * 100))
    print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))

    # Convert binary predictions back to class labels using MultiLabelBinarizer
    predicted_classes = mlb.inverse_transform(y_bin)

    # Use a set to accumulate all distinct classes
    distinct_classes = set()

    # Iterate over predicted classes and add them to the set
    for classes in predicted_classes:
        distinct_classes.update(classes)

    # Convert the set of distinct classes to a sorted list
    class_names = sorted(list(distinct_classes))

    # Print classification report for each class
    print("\nClassification Report:")
    print(classification_report(y_true, y_bin, target_names=class_names))

# Assuming mlb is the MultiLabelBinarizer used for transforming the labels
sklearn_metrics(y_test, y_pred_test, mlb)


261/261 ━━━━━━━━━━━━━━━━━━━━ 232s 887ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 62s 926ms/step
Accuracy        : 88.59
Macro AUC score : 92.18
AUROC           : 80.63
Micro F1 score  : 76.30

Classification Report:
              precision    recall  f1-score   support

          CD       0.84      0.63      0.72       460
         HYP       0.78      0.39      0.52       236
          MI       0.82      0.63      0.71       484
        NORM       0.79      0.93      0.85       918
        STTC       0.80      0.69      0.74       576

   micro avg       0.80      0.73      0.76      2674
   macro avg       0.81      0.66      0.71      2674
weighted avg       0.81      0.73      0.75      2674
 samples avg       0.77      0.74      0.74      2674



d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Test Jupyter\ECG-Classfier-main\myenv2\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len